# Operations problems with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports the construction and solution of optimal power system
scheduling problems (Operations Problems). Opeartions problems form the fundamental
building blocks for [sequential simulations](../../notebook/PowerSimulations_examples/sequential_simulations.ipynb).
This example shows how to specify a the mathematics that will be applied to the data with
an `OperationsProblemTemplate`, build and execute an `OperationsProblem`, and access the results.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using InfrastructureSystems
const IS = InfrastructureSystems
using PowerSystems
const PSY = PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using JuMP
using Cbc #solver

### Data
This data depends upon the [RTS-GMLC](https://github.com/grid-mod/rts-gmlc) dataset. Let's
download and extract the data.

In [5]:
rts_dir = SIIPExamples.download("https://github.com/GridMod/RTS-GMLC")
rts_src_dir = joinpath(rts_dir, "RTS_Data", "SourceData")
rts_siip_dir = joinpath(rts_dir, "RTS_Data", "FormattedData", "SIIP")

"/Users/cbarrows/Documents/repos/Examples/RTS-GMLC-master/RTS_Data/FormattedData/SIIP"

### Create a `System` from RTS-GMLC data just like we did in the [parsing tabular data example.](../../notebook/PowerSystems_examples/parse_tabulardata.jl)

In [6]:
rawsys = PSY.PowerSystemTableData(rts_src_dir,
                                  100.0,
                                  joinpath(rts_siip_dir,"user_descriptors.yaml"),
                                  timeseries_metadata_file = joinpath(rts_siip_dir,"timeseries_pointers.json"),
                                  generator_mapping_file = joinpath(rts_siip_dir,"generator_mapping.yaml"));

sys = System(rawsys; forecast_resolution = Dates.Hour(1));

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages

## Define a problem specification with an `OpModelTemplate`
The `DeviceModel` constructor is to create an assignment between PowerSystems device types
and the subtypes of `AbstractDeviceFormulation`. PowerSimulations has a variety of different
`AbstractDeviceFormulation` subtypes that can be applied to different PowerSystems device types,
each dispatching to different methods for populating optimization problem objectives, variables,
and constraints.

In [7]:
TypeTree(PSI.AbstractDeviceFormulation, scopesep="\n")

PowerSimulations
AbstractDeviceFormulation
├──PowerSimulations
│  AbstractThermalFormulation
│  ├──PowerSimulations
│  │  AbstractThermalDispatchFormulation
│  │  ├──ThermalDispatchNoMin (0 children)
│  │  ├──ThermalDispatch (0 children)
│  │  └──ThermalRampLimited (0 children)
│  └──PowerSimulations
│     AbstractThermalUnitCommitment
│     ├──ThermalBasicUnitCommitment (0 children)
│     └──ThermalStandardUnitCommitment (0 children)
├──PowerSimulations
│  AbstractBranchFormulation
│  ├──PowerSimulations
│  │  AbstractDCLineFormulation
│  │  ├──HVDCDispatch (0 children)
│  │  ├──HVDCLossless (0 children)
│  │  └──PowerSimulations
│  │     VoltageSourceDC (0 children)
│  ├──PowerSimulations
│  │  AbstractLineFormulation
│  │  ├──PowerSimulations
│  │  │  FlowMonitoredLine (0 children)
│  │  ├──StaticLineUnbounded (0 children)
│  │  └──StaticLine (0 children)
│  └──PowerSimulations
│     AbstractTransformerFormulation
│     ├──PowerSimulations
│     │  PhaseControl (0 children)
│     ├──PowerSimulations
│     │  StaticTransformerUnbounded (0 children)
│     ├──StaticTransformer (0 children)
│     └──TapControl (0 children)
├──PowerSimulations
│  AbstractLoadFormulation
│  ├──PowerSimulations
│  │  AbstractControllablePowerLoadFormulation
│  │  ├──DispatchablePowerLoad (0 children)
│  │  └──InterruptiblePowerLoad (0 children)
│  └──StaticPowerLoad
├──PowerSimulations
│  AbstractRenewableFormulation
│  ├──RenewableFixed
│  └──PowerSimulations
│     AbstractRenewableDispatchFormulation
│     ├──RenewableConstantPowerFactor (0 children)
│     └──RenewableFullDispatch (0 children)
├──PowerSimulations
│  AbstractStorageFormulation
│  ├──BookKeeping
│  └──BookKeepingwReservation
└──PowerSimulations
   AbstractHydroFormulation
   ├──PowerSimulations
   │  AbstractHydroDispatchFormulation
   │  ├──PowerSimulations
   │  │  AbstractHydroReservoirFormulation (2 children)
   │  └──HydroDispatchRunOfRiver (0 children)
   ├──HydroFixed
   └──PowerSimulations
      AbstractHydroUnitCommitment

### Branch Formulations
Here is an example of relatively standard branch formulations. Other formulations allow
for selective enforcement of transmission limits and greater control on transformer settings.

In [8]:
branches = Dict{Symbol, DeviceModel}(:L => DeviceModel(Line, StaticLine),
                                     :T => DeviceModel(Transformer2W, StaticTransformer),
                                     :TT => DeviceModel(TapTransformer , StaticTransformer))

Dict{Symbol,DeviceModel} with 3 entries:
  :T  => DeviceModel{Transformer2W,StaticTransformer}(Transformer2W, StaticTran…
  :TT => DeviceModel{TapTransformer,StaticTransformer}(TapTransformer, StaticTr…
  :L  => DeviceModel{Line,StaticLine}(Line, StaticLine, nothing, ServiceModel[])

### Injection Device Formulations
Here we define dictionary entries for all devices that inject or withdraw power on the
network. For each device type, we can define a distinct `AbstractDeviceFormulation`. In
this case, we're defining a basic unit commitment model for thermal generators,
curtailable renewable generators, and fixed dispatch (net-load reduction) formulations
for `HydroFix` and `RenewableFix` devices. Additionally, we've enabled a simple load
shedding demand respons formulation for `InterruptableLoad` devices.

In [9]:
devices = Dict(:Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
                                    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
                                    :Loads =>  DeviceModel(PowerLoad, StaticPowerLoad),
                                    :HydroROR => DeviceModel(HydroDispatch, HydroFixed),
                                    :RenFx => DeviceModel(RenewableFix, RenewableFixed),
                                    :ILoads =>  DeviceModel(InterruptibleLoad, InterruptiblePowerLoad),
                                    )

Dict{Symbol,DeviceModel} with 6 entries:
  :ILoads     => DeviceModel{InterruptibleLoad,InterruptiblePowerLoad}(Interrup…
  :HydroROR   => DeviceModel{HydroDispatch,HydroFixed}(HydroDispatch, HydroFixe…
  :Generators => DeviceModel{ThermalStandard,ThermalStandardUnitCommitment}(The…
  :Ren        => DeviceModel{RenewableDispatch,RenewableFullDispatch}(Renewable…
  :Loads      => DeviceModel{PowerLoad,StaticPowerLoad}(PowerLoad, StaticPowerL…
  :RenFx      => DeviceModel{RenewableFix,RenewableFixed}(RenewableFix, Renewab…

### Service Formulations
We have two `VariableReserve` types, parameterized by their direction. So, similar to
creating `DeviceModel`s, we can create `ServiceModel`s. The primary difference being
that `DeviceModel` objects define how constraints get created, while `ServiceModel` objects
define how constraints get modified.

In [10]:
services = Dict(:ReserveUp => ServiceModel(VariableReserve{ReserveUp}, RangeReserve),
                :ReserveDown => ServiceModel(VariableReserve{ReserveDown}, RangeReserve))

Dict{Symbol,ServiceModel{D,RangeReserve} where D<:Service} with 2 entries:
  :ReserveDown => ServiceModel{VariableReserve{ReserveDown},RangeReserve}(Varia…
  :ReserveUp   => ServiceModel{VariableReserve{ReserveUp},RangeReserve}(Variabl…

### Wrap it up into an `OperationsProblemTemplate`

In [11]:
template_uc= OperationsProblemTemplate(CopperPlatePowerModel, devices, branches, services);

## `OperationsProblem`
Now that we have a `System` and an `OperationsProblemTemplate`, we can put the two together
to create an `OperationsProblem` that we solve.

### Optimizer
It's most convienent to define an optimizer instance upfront and pass it into the
`OperationsProblem` constructor. For this example, we can use the free Cbc solver with a
relatively relaxed MIP gap (`ratioGap`) setting to improve speed.

In [12]:
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Build an `OperationsProblem`
The construction of an `OperationsProblem` essentially applies an `OperationsProblemTemplate`
to `System` data to create a JuMP model.

In [13]:
op_problem = OperationsProblem(GenericOpProblem,
                               template_uc,
                               sys;
                               optimizer = solver,
                               horizon = 12)

┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/devices_models/devices/thermal_generation.jl:369
┌ Info: Generator 202_CT_2 has a nonbinding time limits. Constraints Skipped
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/devices_models/devices/thermal_generation.jl:435
┌ Info: Generator 302_CT_1 has a nonbinding time limits. Constraints Skipped
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/TTXVL/src/devices_models/devices/thermal_generation.jl:435
┌ Info: Generator 202_CT_1 has a nonbinding time limits. Constraints Skipped
└ @ PowerSimulations /Users


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroFixed
      Generators:
        device_type = ThermalStandard
        formulation = ThermalStandardUnitCommitment
      Ren:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RenFx:
        device_type = RenewableFix
        formulation = RenewableFixed
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
  services: 
      ReserveDown:
        service_type = VariableReserve{ReserveDown}
        formulation =

The principal component of the `OperationsProblem` is the JuMP model. For small problems,
you can inspect it by simply printing it to the screen:
```julia
op_problem.psi_container.JuMPmodel
```

For anything of reasonable size, that will be unmanagable. But you can print to a file:
```julia
f = open("testmodel.txt","w"); print(f,op_problem.psi_container.JuMPmodel); close(f)
```

In additon to the JuMP model, an `OperationsProblem` keeps track of a bunch of metadata
about the problem and some references to pretty nammes for constraints and variables.
All of these details are contained within the `psi_container` field.

In [14]:
#nb
print_struct(typeof(op_problem.psi_container))
#nb

mutable struct PowerSimulations.PSIContainer
    JuMPmodel::AbstractModel
    optimizer_factory::Union{Nothing, MathOptInterface.OptimizerWithAttributes}
    time_steps::UnitRange{Int64}
    resolution::TimePeriod
    use_forecast_data::Bool
    initial_time::DateTime
    variables::Dict{Symbol,JuMP.Containers.DenseAxisArray}
    constraints::Dict{Symbol,JuMP.Containers.DenseAxisArray}
    cost_function::AbstractJuMPScalar
    expressions::Dict{Symbol,JuMP.Containers.DenseAxisArray}
    parameters::Union{Nothing, Dict{Symbol,PowerSimulations.ParameterContainer}}
    initial_conditions::Dict{PowerSimulations.ICKey,Array{InitialCondition,N} where N}
    pm::Union{Nothing, PowerModels.AbstractPowerModel}
end


### Solve an `OperationsProblem`

In [15]:
#nb
res = solve_op_problem!(op_problem);
#nb

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 305848 - 0.12 seconds
Cgl0003I 207 fixed, 0 tightened bounds, 653 strengthened rows, 48 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 745 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 633 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 512 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 288 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 219 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 173 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 144 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 135 strengthened rows, 0 substitutions
Cgl0004I processed model has 5694 rows, 11881 colum

## Results Inspection
PowerSimulations collects the `OperationsProblem` results into a struct:

In [16]:
#nb
print_struct(PSI.SimulationResults)
#nb

 struct PowerSimulations.SimulationResults
    variables::Dict{Symbol,DataFrame}
    total_cost::Dict
    optimizer_log::Dict
    time_stamp::DataFrame
    results_folder::Union{Nothing, String}
end


### Optimizer Log
The optimizer summary is included

In [17]:
#nb
res.optimizer_log
#nb

Dict{Symbol,Any} with 9 entries:
  :timed_solve_time   => 7.69083
  :solve_bytes_alloc  => 60162144
  :solve_time         => "Not Supported by COIN Branch-and-Cut (Cbc)"
  :obj_value          => 3.08696e5
  :solver             => "COIN Branch-and-Cut (Cbc)"
  :sec_in_gc          => 0.047653
  :dual_status        => NO_SOLUTION
  :primal_status      => FEASIBLE_POINT
  :termination_status => OPTIMAL

### Total Cost (objective function value)

In [18]:
#nb
res.total_cost
#nb

Dict{Symbol,Float64} with 1 entry:
  :OBJECTIVE_FUNCTION => 3.08696e5

### Variable Values
The solution value data frames for variable in the `op_problem.psi_container.variables`
dictionary is stored:

In [19]:
#nb
res.variables
#nb

Dict{Symbol,DataFrame} with 12 entries:
  Symbol("Spin_Up_R1_Varia… => 12×34 DataFrame. Omitted printing of 28 columns…
  :Start_ThermalStandard    => 12×76 DataFrame. Omitted printing of 71 columns…
  Symbol("Flex_Up_Variable… => 12×102 DataFrame. Omitted printing of 96 columns…
  :Stop_ThermalStandard     => 12×76 DataFrame. Omitted printing of 71 columns…
  Symbol("Flex_Down_Variab… => 12×102 DataFrame. Omitted printing of 96 columns…
  Symbol("Spin_Up_R3_Varia… => 12×43 DataFrame. Omitted printing of 37 columns…
  :P_RenewableDispatch      => 12×30 DataFrame. Omitted printing of 24 columns…
  Symbol("Reg_Down_Variabl… => 12×102 DataFrame. Omitted printing of 96 columns…
  Symbol("Spin_Up_R2_Varia… => 12×25 DataFrame. Omitted printing of 19 columns…
  :P_ThermalStandard        => 12×76 DataFrame. Omitted printing of 71 columns…
  :On_ThermalStandard       => 12×76 DataFrame. Omitted printing of 71 columns…
  Symbol("Reg_Up_VariableR… => 12×102 DataFrame. Omitted printing of 96 colum

For example, we can look at the values for the `:P_ThermalStandard`

In [20]:
#nb
res.variables[:P_ThermalStandard]
#nb

,322_CT_6,321_CC_1,202_STEAM_3,315_STEAM_1,223_CT_4,123_STEAM_2,213_CT_1,223_CT_6
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
2,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
3,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
4,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
5,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
6,0.0,0.0,0.3,0.0,0.0,0.93,0.0,0.0
7,0.0,0.0,0.606667,0.0,0.0,1.24,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.93,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.880389,0.0,0.0


Note that the time stamps are missing...

The time stamps for each value in the time series used in the `OperationsProblem` is
included seperately from the variable value results.

In [21]:
#nb
res.time_stamp
#nb

,Range
,DateTime
1,2020-01-01T00:00:00
2,2020-01-01T01:00:00
3,2020-01-01T02:00:00
4,2020-01-01T03:00:00
5,2020-01-01T04:00:00
6,2020-01-01T05:00:00
7,2020-01-01T06:00:00
8,2020-01-01T07:00:00
9,2020-01-01T08:00:00


## Plotting
PowerSimulaitons also provides some basic specifications for plotting `SimulationResults`.

The plotting capabilities depend on the Julia Plots package.

In [22]:
using PowerGraphics
using Plots
plotly();
#nb

### Bar Plots
We can create a stacked bar plot for any combination of variables to summarize values over
all time periods.

In [23]:
#nb
bar_plot(res, [:P_ThermalStandard])
#nb

### Stack Plots
Similarly, we can create a stack plot for any combination of variable to see the time
series values.

In [24]:
#nb

```stack_plot(res, [:P_ThermalStandard,:P_RenewableDispatch])```

In [25]:
#nb

Or, we can create a series of stack plots for every variable in the dictionary:
```julia
stack_plot(res)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*